In [1]:
import os
import sys
import time
import numpy as np
import pandas as pd
from datetime import date
from datetime import timedelta

START_TIME = time.time()

BASE_URL = "https://datasets.imdbws.com/"
PARQ_PATH = "data/imdb/parquet/"
DOWNLOAD_PATH = "data/imdb/download/"


FILES_IMDB = {
    "cast_crew" : "title.crew.tsv",
    "tit_bas" : "title.basics.tsv",
    "tit_rate" : "title.ratings.tsv",
    "name_bas" : "name.basics.tsv",
    "tit_prin" : "title.principals.tsv",
}

FILES_IMDB_PARQ = {
    "tit_bas" : "title_basics.parquet",
    "genres" : "genres.parquet",
    "tit_rate" : "title_ratings.parquet",
    'directors' : 'directors.parquet',
    'writers' : 'writers.parquet',
    'prim_prof' : 'primary_profession.parquet',
    'known_for' : 'known_for_titles.parquet',
    'name_bas' : 'name_basics.parquet',
    'const' : 'ids.parquet',
    'ordering' : 'ordering.parquet',
    'character' : 'character.parquet',
    'job' : 'job.parquet',
}

FILES_HAND = {
    "add_seen": "add_movies_seen.txt",
    "add_unseen": "add_movies_unseen.txt",
    "add_secop": "add_movies_second_opinion.txt",
    "raw_status": "raw_status.xlsx"
}

FILES_GENERATED = {
    "films_raw": "films_raw.pkl",
    "films_reading": "films_reading.xlsx"
}

In [2]:
ids_and_status = os.path.join("data", "handcrafted", FILES_HAND["raw_status"])
movie_list_raw = pd.read_excel(ids_and_status)

In [3]:
movie_list_raw['watched_date'].dt.date

0             NaT
1      2023-03-04
2             NaT
3             NaT
4             NaT
          ...    
621           NaT
622           NaT
623           NaT
624           NaT
625           NaT
Name: watched_date, Length: 626, dtype: object

# 1) Let's add the integer id's for the movies

In [4]:
const_path = os.path.join(PARQ_PATH, FILES_IMDB_PARQ["const"])
const = pd.read_parquet(const_path)
const.head()

,intid,type
stringid,,
tt0000001,1,tconst
tt0000002,2,tconst
tt0000003,3,tconst
tt0000004,4,tconst
tt0000005,5,tconst


In [5]:
watched_movies = pd.merge(movie_list_raw, const[const["type"]=="tconst"]["intid"], how='left', left_on="tconst", right_index=True)
watched_movies = watched_movies.set_index('intid', drop=True)
del const
watched_movies.head()

,tconst,watched,watched_date,netflix,prime,enjoyment,priority
intid,,,,,,,
15324,tt0015324,0,NaT,NaN,NaN,NaN,NaN
17136,tt0017136,1,2023-03-04,0.0,0.0,3.5,NaN
22100,tt0022100,0,NaT,0.0,0.0,NaN,NaN
25316,tt0025316,0,NaT,NaN,NaN,NaN,NaN
31381,tt0031381,0,NaT,NaN,NaN,NaN,NaN


# 2) lets ad title basics

In [6]:
tit_bas_path = os.path.join(PARQ_PATH, FILES_IMDB_PARQ["tit_bas"])
tit_bas = pd.read_parquet(tit_bas_path)
tit_bas.head()

,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes
tconst,,,,,,,
1,short,Carmencita,Carmencita,False,1894,<NA>,1
2,short,Le clown et ses chiens,Le clown et ses chiens,False,1892,<NA>,5
3,short,Pauvre Pierrot,Pauvre Pierrot,False,1892,<NA>,4
4,short,Un bon bock,Un bon bock,False,1892,<NA>,12
5,short,Blacksmith Scene,Blacksmith Scene,False,1893,<NA>,1


In [7]:
watched_movies = pd.merge(watched_movies, tit_bas, how='left', left_index=True, right_index=True)
del tit_bas
watched_movies.head()

,tconst,watched,watched_date,netflix,prime,enjoyment,priority,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes
intid,,,,,,,,,,,,,,
15324,tt0015324,0,NaT,NaN,NaN,NaN,NaN,movie,Sherlock Jr.,Sherlock Jr.,False,1924,<NA>,45
17136,tt0017136,1,2023-03-04,0.0,0.0,3.5,NaN,movie,Metropolis,Metropolis,False,1927,<NA>,153
22100,tt0022100,0,NaT,0.0,0.0,NaN,NaN,movie,M,M - Eine Stadt sucht einen Mörder,False,1931,<NA>,117
25316,tt0025316,0,NaT,NaN,NaN,NaN,NaN,movie,It Happened One Night,It Happened One Night,False,1934,<NA>,105
31381,tt0031381,0,NaT,NaN,NaN,NaN,NaN,movie,Gone with the Wind,Gone with the Wind,False,1939,<NA>,238


# 3) add rating

In [8]:
rating_path = os.path.join(PARQ_PATH, FILES_IMDB_PARQ["tit_rate"])
tit_rate = pd.read_parquet(rating_path)
tit_rate.head()

,averageRating,numVotes
tconst,,
1,5.7,1968
2,5.8,263
3,6.5,1814
4,5.6,178
5,6.2,2611


In [9]:
watched_movies = pd.merge(watched_movies, tit_rate, how='left', left_index=True, right_index=True)
del tit_rate
watched_movies.head()

,tconst,watched,watched_date,netflix,prime,enjoyment,priority,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,averageRating,numVotes
intid,,,,,,,,,,,,,,,,
15324,tt0015324,0,NaT,NaN,NaN,NaN,NaN,movie,Sherlock Jr.,Sherlock Jr.,False,1924,<NA>,45,8.2,52126.0
17136,tt0017136,1,2023-03-04,0.0,0.0,3.5,NaN,movie,Metropolis,Metropolis,False,1927,<NA>,153,8.3,177829.0
22100,tt0022100,0,NaT,0.0,0.0,NaN,NaN,movie,M,M - Eine Stadt sucht einen Mörder,False,1931,<NA>,117,8.3,161309.0
25316,tt0025316,0,NaT,NaN,NaN,NaN,NaN,movie,It Happened One Night,It Happened One Night,False,1934,<NA>,105,8.1,106925.0
31381,tt0031381,0,NaT,NaN,NaN,NaN,NaN,movie,Gone with the Wind,Gone with the Wind,False,1939,<NA>,238,8.2,321712.0


In [13]:
watched_movies.sort_values(by=['watched','averageRating'], ascending=False)

,tconst,watched,watched_date,netflix,prime,enjoyment,priority,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,averageRating,numVotes
intid,,,,,,,,,,,,,,,,
111161,tt0111161,1,NaT,NaN,NaN,NaN,NaN,movie,The Shawshank Redemption,The Shawshank Redemption,False,1994,<NA>,142,9.3,2735546.0
68646,tt0068646,1,NaT,NaN,NaN,NaN,NaN,movie,The Godfather,The Godfather,False,1972,<NA>,175,9.2,1902341.0
50083,tt0050083,1,NaT,NaN,NaN,4.0,NaN,movie,12 Angry Men,12 Angry Men,False,1957,<NA>,96,9.0,809625.0
167260,tt0167260,1,NaT,NaN,NaN,NaN,NaN,movie,The Lord of the Rings: The Return of the King,The Lord of the Rings: The Return of the King,False,2003,<NA>,201,9.0,1881562.0
468569,tt0468569,1,NaT,NaN,NaN,NaN,NaN,movie,The Dark Knight,The Dark Knight,False,2008,<NA>,152,9.0,2708752.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1462764,tt1462764,0,NaT,NaN,NaN,NaN,NaN,movie,Indiana Jones and the Dial of Destiny,Indiana Jones and the Dial of Destiny,False,2023,<NA>,142,NaN,NaN
15239678,tt15239678,0,NaT,NaN,NaN,NaN,NaN,movie,Dune: Part Two,Dune: Part Two,False,2023,<NA>,<NA>,NaN,NaN
15398776,tt15398776,0,NaT,0.0,0.0,NaN,1.0,movie,Oppenheimer,Oppenheimer,False,2023,<NA>,<NA>,NaN,NaN


In [34]:
watched = watched_movies.query('watched==1')
watched_no_date = watched[watched['watched_date'].isna()].sort_values('averageRating', ascending=False)
watched_date = watched[~watched['watched_date'].isna()].sort_values('watched_date', ascending=False)
not_watched = watched_movies.query('watched==0').sort_values('averageRating', ascending=False)
test = pd.concat([not_watched, watched_date, watched_no_date])
test

,tconst,watched,watched_date,netflix,prime,enjoyment,priority,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,averageRating,numVotes
intid,,,,,,,,,,,,,,,,
108052,tt0108052,0,NaT,1.0,0.0,NaN,NaN,movie,Schindler's List,Schindler's List,False,1993,<NA>,195,9.0,1380976.0
103064,tt0103064,0,NaT,NaN,NaN,NaN,NaN,movie,Terminator 2: Judgment Day,Terminator 2: Judgment Day,False,1991,<NA>,137,8.6,1119827.0
245429,tt0245429,0,NaT,NaN,NaN,NaN,NaN,movie,Spirited Away,Sen to Chihiro no kamikakushi,False,2001,<NA>,125,8.6,786078.0
114814,tt0114814,0,NaT,NaN,NaN,NaN,NaN,movie,The Usual Suspects,The Usual Suspects,False,1995,<NA>,106,8.5,1101781.0
47396,tt0047396,0,NaT,NaN,NaN,NaN,NaN,movie,Rear Window,Rear Window,False,1954,<NA>,112,8.5,500594.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239395,tt0239395,1,NaT,NaN,NaN,NaN,NaN,movie,Cats & Dogs,Cats & Dogs,False,2001,<NA>,87,5.1,60933.0
118747,tt0118747,1,NaT,NaN,NaN,2.0,NaN,movie,Blues Brothers 2000,Blues Brothers 2000,False,1998,<NA>,123,4.9,34010.0
240515,tt0240515,1,NaT,NaN,NaN,NaN,NaN,movie,Freddy Got Fingered,Freddy Got Fingered,False,2001,<NA>,87,4.6,49726.0


,tconst,watched,watched_date,netflix,prime,enjoyment,priority,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,averageRating,numVotes
intid,,,,,,,,,,,,,,,,
108052,tt0108052,0,NaT,1.0,0.0,NaN,NaN,movie,Schindler's List,Schindler's List,False,1993,<NA>,195,9.0,1380976.0
103064,tt0103064,0,NaT,NaN,NaN,NaN,NaN,movie,Terminator 2: Judgment Day,Terminator 2: Judgment Day,False,1991,<NA>,137,8.6,1119827.0
245429,tt0245429,0,NaT,NaN,NaN,NaN,NaN,movie,Spirited Away,Sen to Chihiro no kamikakushi,False,2001,<NA>,125,8.6,786078.0
114814,tt0114814,0,NaT,NaN,NaN,NaN,NaN,movie,The Usual Suspects,The Usual Suspects,False,1995,<NA>,106,8.5,1101781.0
47396,tt0047396,0,NaT,NaN,NaN,NaN,NaN,movie,Rear Window,Rear Window,False,1954,<NA>,112,8.5,500594.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1462764,tt1462764,0,NaT,NaN,NaN,NaN,NaN,movie,Indiana Jones and the Dial of Destiny,Indiana Jones and the Dial of Destiny,False,2023,<NA>,142,NaN,NaN
15239678,tt15239678,0,NaT,NaN,NaN,NaN,NaN,movie,Dune: Part Two,Dune: Part Two,False,2023,<NA>,<NA>,NaN,NaN
15398776,tt15398776,0,NaT,0.0,0.0,NaN,1.0,movie,Oppenheimer,Oppenheimer,False,2023,<NA>,<NA>,NaN,NaN


# 4) add genre

In [10]:
genre_path = os.path.join(PARQ_PATH, FILES_IMDB_PARQ["genres"])
genre = pd.read_parquet(genre_path)
genre.head()

,genre
tconst,
1,Documentary
1,Short
2,Animation
2,Short
3,Animation


efficienter maken

In [11]:
genre = genre[genre.index.isin(watched_movies.index)]

In [12]:
genre['genre'] = genre['genre'].astype('category')

## convert the genres to columns

<a href="https://datascience.stackexchange.com/questions/94038/how-to-create-multi-hot-encoding-from-a-list-column-in-dataframe">How to do multiple-hot easy</a>

<a href="https://stackoverflow.com/questions/58801867/how-to-implement-multilabelbinarizer-on-this-dataframe">Deze nog even bekijken</a>

# kolom naar multi one-hot = pd.crosstab()

In [13]:
genre = pd.crosstab(genre.index, genre['genre'])

In [14]:
watched_movies = pd.merge(watched_movies, genre, how='left', left_index=True, right_index=True)

In [15]:
del genre

In [16]:
watched_movies

,tconst,watched,watched_date,netflix,prime,enjoyment,priority,titleType,primaryTitle,originalTitle,...,Music,Musical,Mystery,Romance,Sci-Fi,Short,Sport,Thriller,War,Western
intid,,,,,,,,,,,,,,,,,,,,,
15324,tt0015324,0,NaT,NaN,NaN,NaN,NaN,movie,Sherlock Jr.,Sherlock Jr.,...,0,0,0,1,0,0,0,0,0,0
17136,tt0017136,1,2023-03-04,0.0,0.0,3.5,NaN,movie,Metropolis,Metropolis,...,0,0,0,0,1,0,0,0,0,0
22100,tt0022100,0,NaT,0.0,0.0,NaN,NaN,movie,M,M - Eine Stadt sucht einen Mörder,...,0,0,1,0,0,0,0,1,0,0
25316,tt0025316,0,NaT,NaN,NaN,NaN,NaN,movie,It Happened One Night,It Happened One Night,...,0,0,0,1,0,0,0,0,0,0
31381,tt0031381,0,NaT,NaN,NaN,NaN,NaN,movie,Gone with the Wind,Gone with the Wind,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9691136,tt9691136,0,NaT,0.0,0.0,NaN,NaN,movie,Shadow in the Cloud,Shadow in the Cloud,...,0,0,0,0,0,0,0,0,1,0
9731598,tt9731598,0,NaT,0.0,0.0,NaN,NaN,movie,Bros,Bros,...,0,0,0,1,0,0,0,0,0,0
9764362,tt9764362,1,NaT,0.0,0.0,4.0,NaN,movie,The Menu,The Menu,...,0,0,0,0,0,0,0,1,0,0


In [17]:
# watched_movies = pd.merge(watched_movies, genre, how='left', left_index=True, right_index=True)
# del genre
# watched_movies.head()

In [18]:
# watched_movies['genre'] = watched_movies['genre'].astype('category')
# genre = pd.crosstab(watched_movies.index, watched_movies['genre'])
# watched_movies = watched_movies.drop(columns='genre').drop_duplicates()

In [19]:
# watched_movies = pd.merge(watched_movies, genre, how='left', left_index=True, right_index=True)
# del genre
# watched_movies

In [20]:
# watched_movies

# 5) new add directors / writers / other personell to ordering list

convert directors and writers to tconst(index) - nconst - category

In [21]:
directors_path = os.path.join(PARQ_PATH, FILES_IMDB_PARQ["directors"])
directors = pd.read_parquet(directors_path)
directors = directors[directors.index.isin(watched_movies.index)]
directors['category'] = 'director'
directors = directors.rename(columns={'directors':'nconst'})

In [22]:
directors

,nconst,category
tconst,,
15324,36,director
17136,485,director
22100,485,director
25316,1008,director
31381,281808,director
...,...,...
9691136,1755396,director
9731598,831557,director
9764362,617042,director


In [23]:
writers_path = os.path.join(PARQ_PATH, FILES_IMDB_PARQ["writers"])
writers = pd.read_parquet(writers_path)
writers = writers[writers.index.isin(watched_movies.index)]
writers['category'] = 'writer'
writers = writers.rename(columns={'writers':'nconst'})

In [24]:
writers

,nconst,category
tconst,,
15324,369841,writer
15324,593477,writer
15324,115669,writer
17136,902376,writer
17136,485,writer
...,...,...
9783600,2808422,writer
9783600,1014201,writer
9783600,1116660,writer


In [25]:
ordering_path = os.path.join(PARQ_PATH, FILES_IMDB_PARQ["ordering"])
ordering = pd.read_parquet(ordering_path)
ordering = ordering[ordering.index.isin(watched_movies.index)].drop(columns='ordering')

In [26]:
ordering

,nconst,category
tconst,,
15324,504380,cinematographer
15324,36,actor
15324,570230,actress
15324,444172,actor
15324,175068,actor
...,...,...
9806192,3021346,director
9806192,491011,writer
9806192,238941,producer


In [27]:
personell = pd.concat([writers, directors, ordering])
personell = personell.drop_duplicates()
del writers
del directors
del ordering
personell.loc[:,'category'] = personell.loc[:,'category'].astype('category')

In [28]:
personell

,nconst,category
tconst,,
15324,369841,writer
15324,593477,writer
15324,115669,writer
17136,902376,writer
17136,485,writer
...,...,...
9806192,1253936,actor
9806192,11127862,actor
9806192,238941,producer


In [19]:
# get needed personell info
name_bas_path = os.path.join(PARQ_PATH, FILES_IMDB_PARQ["name_bas"])
name_bas = pd.read_parquet(name_bas_path)
names = name_bas[name_bas.index.isin(personell['nconst'])]
del name_bas

# convert personell info
names = names["primaryName"] + " (" + names["birthYear"].astype(str).replace("<NA>","") + "-" + names["deathYear"].astype(str).replace("<NA>","") + ")"
names.name = "info"

# add personell info to movie indices
personell = pd.merge(personell, names, how='left', left_on='nconst', right_index=True).drop(columns='nconst')
del names

# convert personell categories to columns
personell = pd.pivot_table(personell, values='info', index=['tconst'], columns=['category'], aggfunc=list)

# add personell to watched movies
watched_movies = pd.merge(watched_movies, personell, how='left', left_index=True, right_index=True)
del personell

In [20]:
watched_movies

,tconst,watched,watched_date,netflix,prime,enjoyment,priority,titleType,primaryTitle,originalTitle,...,actress,archive_footage,cinematographer,composer,director,editor,producer,production_designer,self,writer
15324,tt0015324,0,NaT,NaN,NaN,NaN,NaN,movie,Sherlock Jr.,Sherlock Jr.,...,[Kathryn McGuire (1903-1978)],NaN,"[Elgin Lessley (1883-1944), Byron Houck (1891-...",[Club Foot Orchestra (-)],[Buster Keaton (1895-1966)],NaN,NaN,NaN,NaN,"[Jean C. Havez (1872-1925), Joseph A. Mitchell..."
17136,tt0017136,1,2023-03-04,0.0,0.0,3.5,NaN,movie,Metropolis,Metropolis,...,[Brigitte Helm (1906-1996)],NaN,NaN,"[Sandro Forte (1970-2020), Maximianno Cobra (1...",[Fritz Lang (1890-1976)],NaN,[Erich Pommer (1889-1966)],NaN,NaN,"[Thea von Harbou (1888-1954), Fritz Lang (1890..."
22100,tt0022100,0,NaT,0.0,0.0,NaN,NaN,movie,M,M - Eine Stadt sucht einen Mörder,...,"[Ellen Widmann (1894-1985), Inge Landgut (1922...",NaN,[Fritz Arno Wagner (1894-1958)],NaN,NaN,[Paul Falkenberg (1903-1986)],NaN,NaN,NaN,[Egon Jacobsohn (1895-)]
25316,tt0025316,0,NaT,NaN,NaN,NaN,NaN,movie,It Happened One Night,It Happened One Night,...,[Claudette Colbert (1903-1996)],NaN,[Joseph Walker (1892-1985)],NaN,[Frank Capra (1897-1991)],[Gene Havlick (1894-1959)],NaN,NaN,NaN,"[Robert Riskin (1897-1955), Samuel Hopkins Ada..."
31381,tt0031381,0,NaT,NaN,NaN,NaN,NaN,movie,Gone with the Wind,Gone with the Wind,...,"[Vivien Leigh (1913-1967), Barbara O'Neil (191...",NaN,NaN,NaN,"[Victor Fleming (1889-1949), George Cukor (189...",NaN,NaN,NaN,NaN,"[Margaret Mitchell (1900-1949), Sidney Howard ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16118722,tt16118722,0,NaT,0.0,0.0,NaN,NaN,movie,Tony Hawk: Until the Wheels Fall Off,Tony Hawk: Until the Wheels Fall Off,...,NaN,NaN,[Jesse Green (-)],[Jeff Cardoni (-)],[Sam Jones (-)],[Greg Finton (-)],NaN,NaN,"[Tony Hawk (1968-), Stacy Peralta (1957-), Rod...",NaN
17076046,tt17076046,0,NaT,0.0,0.0,NaN,NaN,movie,Weird: The Al Yankovic Story,Weird: The Al Yankovic Story,...,NaN,NaN,NaN,NaN,[Eric Appel (1980-)],NaN,"[Tim Headington (-), Lia Buman (-), Mike Farah...",NaN,NaN,"['Weird Al' Yankovic (1959-), Eric Appel (1980-)]"
19719904,tt19719904,0,NaT,0.0,0.0,NaN,NaN,movie,Plan 75,Plan 75,...,"[Chieko Baishô (1941-), Stefanie Arianne (1994-)]",NaN,NaN,[Rémi Boubal (-)],[Chie Hayakawa (1976-)],NaN,"[Frédéric Corvez (-), Eiko Mizuno Gray (-), Ma...",NaN,NaN,"[Jason Gray (-), Chie Hayakawa (1976-)]"
20225374,tt20225374,0,NaT,0.0,0.0,NaN,NaN,movie,Vesper,Vesper,...,"[Raffiella Chapman (2007-), Rosy McEwen (-)]",NaN,NaN,NaN,"[Kristina Buozyte (1982-), Bruno Samper (-)]",NaN,"[Alexis Perrin (-), Daiva Jovaisiene (-), Asta...",NaN,NaN,"[Bruno Samper (-), Kristina Buozyte (1982-), B..."
